# 1. Notebook Setup
# Title, assignment info, and markdown overview.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rd
import csv
import math

In [ ]:
# 2. Imports


class Task:
    time:int =0
    difficulty:int =0
    deadline:int = 0
    required_skill:str = ''
    def __str__(self):
        return f"T Time:{self.time} Difficulty:{self.difficulty} Deadline:{self.deadline} requiredSkill:{self.required_skill}"

class Employee:
    available_hours:int = 0
    skill_level:int = 4
    skills:list[str]=['']

    def __str__(self):
        return f"E AH:{self.available_hours} Skill-Level:{self.skill_level} Skills:{self.skills}"

TDMapping = list[list[int]]

# ask ID Time (hrs) Difficulty Deadline (hrs) Required Skill
# T1 4 3 8 A
# T2 6 5 12 B
# T3 2 2 6 A
# T4 5 4 10 C
# T5 3 1 7 A# ask ID Time (hrs) Difficulty Deadline (hrs) Required Skill
# T1 4 3 8 A
# T2 6 5 12 B
# T3 2 2 6 A
# T4 5 4 10 C
# T5 3 1 7 A
# T6 8 6 15 B
# T7 4 3 9 C
# T8 7 5 14 B
# T9 2 2 5 A
# T10 6 4 11 C
# T6 8 6 15 B
# T7 4 3 9 C
# T8 7 5 14 B
# T9 2 2 5 A
# T10 6 4 11 C


In [ ]:
# I/O
"""
Task csv:

'ID', 'Time (hrs)', 'Difficulty', 'Deadline (hrs)', 'Required Skill'

Employee csv:
'Employee ID', 'Available Hrs', 'Skill Level', 'Skills'


Using an adjancey list instead of an adjacney matrix!

Since the input vector is 10 mappings x 11 features. Including a unique assignment penalty is redundant since a task mapped to 2 employees will create (10+1) mappings which doesn't work for the network input layer!
"""





class DataLoader:
    num_of_tasks =10
    num_of_employees = 5
    tasks:list[Task]=[]
    employees:list[Employee]=[]
    employees_file_path = "data/Employee.csv"
    tasks_file_path = "data/Tasks.csv"
    def load_tasks(self):
        tasks =[]
        with open(self.tasks_file_path,'r') as csvfile:
            taskReader = csv.reader(csvfile)
            next(taskReader)
            for taskArr in taskReader:
                newT = Task()
                newT.time=int(taskArr[1])
                newT.difficulty= int(taskArr[2])
                newT.deadline = int(taskArr[3])
                newT.required_skill= taskArr[4]
                tasks.append(newT)
        return tasks

    def load_employees(self):
        employees=[]
        with open(self.employees_file_path,'r') as csvfile:
            employeeReader = csv.reader(csvfile)
            next(employeeReader)
            for employeeArr in employeeReader:
                newEmployee = Employee()
                newEmployee.available_hours=int(employeeArr[1])
                newEmployee.skill_level= int(employeeArr[2])
                skills = employeeArr[3].split(',')
                newEmployee.skills = skills
                employees.append(newEmployee)
        return employees
    def loadAll(self):
        self.load_employees()
        self.load_tasks()
    


class MappingHandler:
    mapping_file_path = "data/Mappings.csv"

    def __init__(self,tasks:list[Task],employees:list[Employee],num_of_mappings=100):
        self.tasks:list[Task]= tasks
        self.employees:list[Employee] = employees
        self.num_of_mappings = num_of_mappings




    def __costFunction(self,Mapping:TDMapping):
        w=0.2
        overload = 0
        skill_mismatch = 0
        difficulty_violation = 0
        deadline_violation = 0
        unique_assignment = 0
        employee_task_adj_list:list[list[Task]]=[[] for _ in range(len(self.employees))]

        for i in range(len(Mapping)):
            taskId=i
            taskMapping = Mapping[i]
            task = self.tasks[taskId]
            num_of_employees_assigned = 0
            for employeeId in taskMapping:
                num_of_employees_assigned+=1
                employee = self.employees[employeeId]
                employee_task_adj_list[employeeId].append(task)
                # skill mismatch violation
                if task.required_skill not in employee.skills:
                    skill_mismatch+=1
                if task.difficulty> employee.skill_level:
                    difficulty_violation+= task.difficulty-employee.skill_level
            unique_assignment+= max(0,num_of_employees_assigned-1)



        for employeeId in range(len(employee_task_adj_list)):
            employee = self.employees[employeeId]
            sortedEmployeeTasks = employee_task_adj_list[employeeId]
            sortedEmployeeTasks.sort(key= lambda x:x.time)
            sumHours=0
            finishTime=0
            for t in sortedEmployeeTasks:
                sumHours+=t.time
                finishTime+= t.time
                deadline_violation+= max(0,finishTime - t.deadline)
            overload+= max(0,sumHours- employee.available_hours) 
        total_penalty = overload+skill_mismatch+unique_assignment+deadline_violation+ difficulty_violation
        print(overload)
        return round(total_penalty*w,3)



    def __createMappings(self):
        unique = set()
        all_mappings=[]
        while len(unique) < self.num_of_mappings:
            mapping:TDMapping = [[] for _ in self.tasks]         # Create a mapping for each iteration. Using an adjlist format
            # possible_assignments = [i for i in range(len(employees))]  #list of possible assignments
            for taskId in range(len(self.tasks)):
                rand_employee =rd.randint(0,len(self.employees)-1)
                mapping[taskId].append(rand_employee)
            string = str(mapping)
            if string not in unique: # ensure no duplicates in data generation
                unique.add(string)
                all_mappings.append(mapping)
        return all_mappings
    

    def __mappingToCsv(self,mappings:list[TDMapping]):
        with open(self.mapping_file_path,'w') as csvfile:
                mappingWriter = csv.writer(csvfile)
                mappingWriter.writerow(["T1",'T2','T3','T4','T5','T6','T7','T8','T9','T10','Penalty'])
                for i in range(len(mappings)):
                    flattened=[]
                    for lst in mappings[i]:
                        for e in lst:
                            flattened.append(e)
                    flattened.append(self.__costFunction(mappings[i]))
                    mappingWriter.writerows(flattened)
        
    def generateMappings(self):
        all_mappings = self.__createMappings()
        # self.__mappingToCsv(all_mappings)




def testDataGen():
    dl = DataLoader()
    dl.loadAll()
    mappingloader = MappingHandler(dl.tasks,dl.employees)
    mappingloader.generateMappings()

testDataGen()


# def TestDataGen():

#     all_mappings = mappingGenerator(task,employees)
#     for mapping in all_mappings:
#         cost = costFunction(mapping,task,employees)
#         print(cost)

# def verifyCost():
#     mapping=[[2],[3],[1],[4],[2],[5],[1],[3],[5],[4]]
    
#     for i in range(len(mapping)):
#         mapping[i][0]-=1
#     employees,tasks = DataLoader().loadAll()
#     # for t in tasks:
#     #     print(t)
    
#     for e in employees:
#         print(e)
#     print(costFunction(mapping,tasks,employees))





do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
do
d

KeyboardInterrupt: 

In [ ]:
#data processing and I/O




def dataLoad():



# 3. Data Generation/Load
#- Generate or load the 100 mappings CSV
#- Load into pandas DataFrame
# 4. Preprocessing
def one_hot_encode_skill(skills):
    pass
# returns a 3-element vector
...
def construct_input_vector(mapping_row):
    pass
# builds 110-dim vector for one example

# 5. Model Definitions
class NeuralNetwork:
    def __init__(self, layer_dims, activation=’relu’):
        pass

    def forward(self, x):
        pass

    def backward(self, x, y_true):
        pass

    def update_params(self, lr):
        pass

    # 6. Training Loop
    def train(model, X_train, y_train, params):
        pass
    # implement mini-batch SGD, record loss

# 7. Evaluation & Plots
#- Generate the eight required figures
#- Save each via plt.savefig()
# 8. Save & Export
#- Download figures
#- Optionally, pickle model parameters